In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

import spacy

In [2]:
df = pd.read_csv("train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [3]:
df['question1'][2]

'How can I increase the speed of my internet connection while using a VPN?'

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [5]:
new_lines=[]
lines = []
for line in list(df['question1']):
    new_lines=line.split(' ')
    lines.append(new_lines)
#new lines has the new format
#lines=new_lines
#lines

[['What',
  'is',
  'the',
  'step',
  'by',
  'step',
  'guide',
  'to',
  'invest',
  'in',
  'share',
  'market',
  'in',
  'india?'],
 ['What', 'is', 'the', 'story', 'of', 'Kohinoor', '(Koh-i-Noor)', 'Diamond?'],
 ['How',
  'can',
  'I',
  'increase',
  'the',
  'speed',
  'of',
  'my',
  'internet',
  'connection',
  'while',
  'using',
  'a',
  'VPN?'],
 ['Why',
  'am',
  'I',
  'mentally',
  'very',
  'lonely?',
  'How',
  'can',
  'I',
  'solve',
  'it?'],
 ['Which',
  'one',
  'dissolve',
  'in',
  'water',
  'quikly',
  'sugar,',
  'salt,',
  'methane',
  'and',
  'carbon',
  'di',
  'oxide?'],
 ['Astrology:',
  'I',
  'am',
  'a',
  'Capricorn',
  'Sun',
  'Cap',
  'moon',
  'and',
  'cap',
  'rising...what',
  'does',
  'that',
  'say',
  'about',
  'me?'],
 ['Should', 'I', 'buy', 'tiago?'],
 ['How', 'can', 'I', 'be', 'a', 'good', 'geologist?'],
 ['When', 'do', 'you', 'use', 'シ', 'instead', 'of', 'し?'],
 ['Motorola',
  '(company):',
  'Can',
  'I',
  'hack',
  'my',
  'Char

In [6]:
new_lines2=[]
lines2 = []
for line in list(df['question2']):
    new_lines=line.split(' ')
    lines2.append(new_lines)
#lines2

In [7]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(lines, min_count=1, size=200)


In [8]:
model2 = Word2Vec(lines2, min_count=1, size=200)

In [9]:
vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])): 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(qu1.split(' ')),200])
    for word1 in qu1.split(' '):
        # word2vec
        vec1 = model.wv[str(word1)]
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    #print(mean_vec1)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)

100%|████████████████████████████████████████████████████████████████████████| 404290/404290 [01:02<00:00, 6429.72it/s]


In [10]:
vecs2 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question2'])): 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(qu1.split(' ')),200])
    for word1 in qu1.split(' '):
        # word2vec
        vec1 = model2.wv[str(word1)]
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    #print(mean_vec1)
    vecs2.append(mean_vec1)
df['q2_feats_m'] = list(vecs2)

100%|████████████████████████████████████████████████████████████████████████| 404290/404290 [00:56<00:00, 7181.26it/s]


In [13]:
dfppro = pd.read_csv("quora_fe.csv",encoding='latin-1')
dfnlp = pd.read_csv("df_text_processed.csv", encoding='latin-1')

In [14]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [15]:
df3_q1['id']=df1['id']
df3_q2['id']=df1['id']
df1  = df1.merge(df2, on='id',how='left')
df2  = df3_q1.merge(df3_q2, on='id',how='left')
result  = df1.merge(df2, on='id',how='left')
result.to_csv('final_features.csv')

In [16]:
result.dtypes

id               object
is_duplicate    float64
cwc_min         float64
cwc_max         float64
csc_min         float64
                 ...   
195_y           float64
196_y           float64
197_y           float64
198_y           float64
199_y           float64
Length: 428, dtype: object